In [1]:
import paho.mqtt.client as mqtt
from time import sleep
import json

In [2]:
import redis

REDIS_HOST = "redis-17243.c250.eu-central-1-1.ec2.cloud.redislabs.com"
REDIS_PORT = 17243
REDIS_USER = "default"
REDIS_PSSW = "IIRJyR2QEWYULfX8g9ZTXAAeG3shxYsm"
redis_client = redis.Redis(REDIS_HOST, REDIS_PORT, password=REDIS_PSSW, username=REDIS_USER,)

In [3]:
redis_client = redis.Redis(REDIS_HOST, REDIS_PORT, password=REDIS_PSSW, username=REDIS_USER,)
print('Is connected:', redis_client.ping())

Is connected: True


In [4]:
client = mqtt.Client() #It used to send or receive message.
sid = 's301665'
#These methods are mandatory and the parameters are defined by paho, so don't change them.
def on_connect(client, userdata, flags, rc):
    '''
    client: mqtt client.
    rc: result code of the connection.
    '''

    print(f'Connected with result code: {str(rc)}')
    #Subscribing to the broker.
    client.subscribe(sid) 

def on_message(client, userdata, msg):
    message = msg.payload.decode()
    topic = msg.topic
    obj = json.loads(message)

    print(f'Received message: {message} to the topic: {topic} ')
    
    mac_address = obj['mac_address']
    timestamp_ms = obj['timestamp']
    prediction = obj['prediction']
    probability = obj['probability']
    latency = obj['latency']
    #if there is a new monitored device, it will create the timeseries.
    try:
        redis_client.ts().create(f'{mac_address}:prediction')
        redis_client.ts().create(f'{mac_address}:probability')
        redis_client.ts().create(f'{mac_address}:latency')
    except:
        pass

    redis_client.ts().add(f'{mac_address}:prediction', timestamp_ms, prediction )
    redis_client.ts().add(f'{mac_address}:probability', timestamp_ms, probability )
    redis_client.ts().add(f'{mac_address}:latency', timestamp_ms, latency )


In [5]:
client.on_connect = on_connect
client.on_message = on_message

client.connect('mqtt.eclipseprojects.io', 1883)

client.loop_forever()


Connected with result code: 0
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584402209, "prediction": 2.0, "probability": 99.73, "latency": 4.327016353607178} to the topic: s301665 
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584409172, "prediction": 2.0, "probability": 99.56, "latency": 3.4959237575531006} to the topic: s301665 
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584415531, "prediction": 2.0, "probability": 99.8, "latency": 2.788954019546509} to the topic: s301665 
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584422752, "prediction": 2.0, "probability": 100.0, "latency": 3.401998996734619} to the topic: s301665 
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584429406, "prediction": 3.0, "probability": 68.37, "latency": 3.113957643508911} to the topic: s301665 
Received message: {"mac_address": "0x2c8db1c3edca", "timestamp": 1686584439994, "prediction": 3.0, "

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05dea53b-e624-4a45-a48c-4f8a58ba823b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>